In [40]:
import math
import numpy as np
import ortools
import itertools

from typing import Generator, List, Tuple

In [68]:
# look at all possible cut assignments that result in the needed quantities
#   minimize over all of the cut assignments
from ortools.sat.python import cp_model


In [47]:
# Toy problem:
needed_lengths = [3, 5, 8]
needed_quantities = [1, 2, 1]
base_length = 10

# We expect to be allowed to make cuts
# 
# allowed_cuts = [
#   (3,), (3, 3,), (3, 3, 3,), (3, 5,), (5,), (5, 3,), (5, 5,), (8,),
# ]

In [48]:
def get_possible_cuts(needed_lengths: List[int], base_length: int) -> Generator:
    
    # Establish a bound on the number of combinations that we need
    max_combinations = math.floor(base_length / min(needed_lengths))

    for i in range(1, max_combinations + 1):
        for comb in itertools.combinations_with_replacement(needed_lengths, i):
            if sum(comb) <= base_length:
                yield comb

In [49]:
possible_cuts = list(
    get_possible_cuts(needed_lengths, base_length)
)

In [85]:
cut_produced_quantities = {}
for i, cut in enumerate(possible_cuts):
    for j, length in enumerate(needed_lengths):
        cut_produced_quantities[i, j] = cut.count(length)
        
cut_wastes = {}
for i, cut in enumerate(possible_cuts):
    cut_wastes[i] = base_length - sum(cut)

In [98]:
model = cp_model.CpModel()
naive_max_cuts = sum(needed_quantities)

cut_vars = {}
for i, cut in enumerate(possible_cuts):
    cut_vars[i] = model.NewIntVar(0, naive_max_cuts, f"Cut according to rule {cut}")

for (j, length), needed in zip(enumerate(needed_lengths), needed_quantities): 
    cut_sums = []
    for i, cut in enumerate(possible_cuts):
        cut_sums.append(cut_vars[i] * cut_produced_quantities[i, j])
    model.Add(sum(cut_sums) == needed)
    
# Let's minimize the cut wastest
total_cut_waste = sum(
    cut_vars[i] * cut_wastes[i] for i, _ in enumerate(possible_cuts)
)
model.Minimize(total_cut_waste)

In [99]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [101]:
for i, var in cut_vars.items():
    print(possible_cuts[i], solver.Value(var))

(3,) 0
(5,) 1
(8,) 1
(3, 3) 0
(3, 5) 1
(5, 5) 0
(3, 3, 3) 0
